<a href="https://colab.research.google.com/github/galymzhan-zharas/Voice-assistant/blob/main/Voice_assisstant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# write LLM text generation
# write a TTS pipeline using OpenAI API
  # replace it with a different, free model later (see how it's done in ASR tutorial in HG face)
# if all works connect them through langchain
  # ask claude and chatgpt for langchain tutorial
  # try connecting it myself
  # send the code to gpt to ask if all good
  # modularize the code (define functions, etc)
# write a GUI

In [1]:
!git config --global user.name "galymzhan-zharas"
!git config --global user.email "galymzhanzharas01@gmail.com"

In [2]:
!git clone https://github.com/galymzhan-zharas/Voice-assistant.git code

Cloning into 'code'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [3]:
%cd code

/content/code


In [4]:
!pip install -qU pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.3/524.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 19.7 MB/s eta 0:00:00


# Pinecone

In [5]:
from google.colab import userdata

In [6]:
# Set your API key and environment
from pinecone import Pinecone

pc = Pinecone(api_key=userdata.get('pinecone-api-key'))

In [7]:
# does it create an index or namespace?
# what does it do?
index_name = 'voice-assistant'

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

In [8]:
import os


In [9]:
# check if a directory called 'documents' exists
if not os.path.exists('/content/documents'):
    # create a directory called 'documents'
    os.makedirs('/content/documents')

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [11]:
# this splitter can be used for paragraph splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,       # ~600 tokens
    chunk_overlap=300,     # overlap to retain context
    separators=["\n\n", "\n", ".", " ", ""]
)

In [12]:
!pip install -q python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.2 MB/s eta 0:00:00


In [13]:
from docx import Document

In [33]:
file_p = '/content/documents/Lizard.docx'
document = Document(file_p)
full_text = []
for i,para in enumerate(document.paragraphs):
    full_text.append(para.text)
    print(i, para.text)

full_text = '\n'.join(full_text).split(' ')
print(len(full_text))

0 Noctiserpens aquavarian: The Shape-Shifting Evolutionary Lizard
1 Noctiserpens aquavarian is a newly discovered, semi-mythical reptilian species exhibiting remarkable adaptive and morphogenetic abilities. First documented in 2023 in the mangrove-laced coastlines of southeastern Papua New Guinea, this unique lizard is known for its unprecedented capacity to transform into a snake-like form during the night and is showing early-stage biological adaptations toward becoming a fully aquatic marine reptile. Biologists consider it a living example of evolution in motion.
2 
3 Taxonomy and Classification
4 	•	Kingdom: Animalia
5 	•	Phylum: Chordata
6 	•	Class: Reptilia
7 	•	Order: Squamata
8 	•	Family: Noctisauridae (proposed)
9 	•	Genus: Noctiserpens
10 	•	Species: N. aquavarian
11 	•	Common Names: Twilight Shifter, Serpent Lizard, Aqua Wyrm
12 
13 Appearance
14 In its diurnal (daytime) form, N. aquavarian resembles a slender, arboreal lizard approximately 50–70 cm in length. Its scales are

In [14]:
# here, I am chunking 3 documents and storing each chunk in the list.
# each chunk is represted as a dictionary that contains the text itself as well as supporting meta-data
records = []
doc_num = 0
for file_name in os.listdir('/content/documents'):
  doc_num +=1
  # ------- for reading text data -------- #

  # with open(os.path.join('/content/documents', file_name), 'r') as f:
  #   text = f.read()
  #   chunks = text_splitter.split_text(text)
  #   for chunk_num, chunk in enumerate(chunks,start=1):

  #     record = {
  #         "_id": f"document{doc_num}#chunk{chunk_num}", # it helps to update or delete specific chunk
  #         'chunk_text': chunk,
  #         'document_id':f"document{doc_num}",
  #         'document_title': file_name,
  #         'chunk_num': chunk_num
  #     }
  #     records.append(record)

  # ------- for reading docx  -------- #
  document = Document(os.path.join('/content/documents', file_name))
  full_text = []
  for para in document.paragraphs:
    full_text.append(para.text)
  text = '\n'.join(full_text)

  chunks = text_splitter.split_text(text)
  for chunk_num, chunk in enumerate(chunks,start=1):
    record = {
        "_id": f"document{doc_num}#chunk{chunk_num}", # it helps to update or delete specific chunk
        'chunk_text': chunk,
        'document_id':f"document{doc_num}",
        'document_title': file_name,
        'chunk_num': chunk_num
    }
    records.append(record)

In [35]:
records

[{'_id': 'document1#chunk1',
  'chunk_text': "Noctiserpens aquavarian: The Shape-Shifting Evolutionary Lizard\nNoctiserpens aquavarian is a newly discovered, semi-mythical reptilian species exhibiting remarkable adaptive and morphogenetic abilities. First documented in 2023 in the mangrove-laced coastlines of southeastern Papua New Guinea, this unique lizard is known for its unprecedented capacity to transform into a snake-like form during the night and is showing early-stage biological adaptations toward becoming a fully aquatic marine reptile. Biologists consider it a living example of evolution in motion.\n\nTaxonomy and Classification\n\t•\tKingdom: Animalia\n\t•\tPhylum: Chordata\n\t•\tClass: Reptilia\n\t•\tOrder: Squamata\n\t•\tFamily: Noctisauridae (proposed)\n\t•\tGenus: Noctiserpens\n\t•\tSpecies: N. aquavarian\n\t•\tCommon Names: Twilight Shifter, Serpent Lizard, Aqua Wyrm\n\nAppearance\nIn its diurnal (daytime) form, N. aquavarian resembles a slender, arboreal lizard approxi

In [15]:
len(records)

12

In [16]:
dense_index = pc.Index(index_name) # just an API for the created index
dense_index.upsert_records("articles", records) # they are upserted into the namespace 'narratives'

In [17]:
# view how many indeces you've upserted
stats = dense_index.describe_index_stats()
print(stats)

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'articles': {'vector_count': 12},
                'narratives': {'vector_count': 6}},
 'total_vector_count': 18,
 'vector_type': 'dense'}


In [43]:
# Define the query
query = "What are the lizard's special abilities?"

# Search the dense index
results = dense_index.search(
    namespace="articles",
    query={
        "top_k": 5,
        "inputs": {
            'text': query
        }
    }
)

# Print the results
for hit in results['result']['hits']:
        print(f"id: {hit['_id']:<5} | score: {round(hit['_score'], 2):<5} | text: {hit['fields']['chunk_text']:<50}")
        print()

id: document1#chunk2 | score: 0.43  | text: Special Abilities
1. Nocturnomorphic Shapeshifting
The most distinctive feature of N. aquavarian is its ability to morph between lizard and snake forms based on circadian rhythms. This shapeshifting is regulated by a unique hormonal compound dubbed morphocortisol, triggered by the absence of UV light and temperature drops.
2. Aquatic Pre-Adaptation
Individuals living in estuarine zones display elongated digits, webbed feet, and denser bone structures. There is growing evidence that N. aquavarian is slowly transitioning into a marine reptile—a rare, real-time example of evolutionary transmutation. Marine populations spend up to 14 hours submerged, using lung-reserve breathing techniques.
3. Photocamouflage
Thanks to iridophores in their dermal layers, N. aquavarian can subtly change color and reflect surrounding light patterns, allowing for effective camouflage both in foliage and shallow reef environments.
4. Neuroadaptive Intelligence
Recent

In [44]:
# Define the query
query = "How old is Galymzhan?"

# Search the dense index
results_reranked = dense_index.search(
    namespace="narratives",
    query={
        "top_k": 10,
        "inputs": {
            'text': query
        }
    },
    rerank={
      "model": "bge-reranker-v2-m3",
      "top_n": 10,
      "rank_fields": ["chunk_text"]
    }
)

# Print the results
for hit in results_reranked['result']['hits']:
        print(f"id: {hit['_id']:<5} | score: {round(hit['_score'], 2):<5} | text: {hit['fields']['chunk_text']:<50}")


id: document2#chunk1 | score: 1.0   | text: Galymzhan Zharas, a spirited 23-year-old from the sunlit plains of Kyzylorda, Kazakhstan, always dreamed of pushing beyond his humble hometown to discover the world. Inspired by ancient Kazakh nomads who roamed endlessly across vast steppes, Galymzhan took this spirit into the modern age, trading horses for horsepower.

One morning, galvanized by an intense desire for adventure, Galymzhan began his journey aboard his faithful companion, an impeccably tuned Kawasaki Ninja H2, affectionately named "Tulpar," after the mythical winged horse of Kazakh folklore. His ambitious plan was nothing less than circling the globe in a staggering timeframe of just one month.

His departure from Kyzylorda was a festive event, cheered on by family and locals who were awed and inspired by his ambition. With the Kazakh flag draped across his shoulders, Galymzhan sped westward, aiming first for Europe, traversing rough roads, serene countrysides, and bustling cit

In [ ]:
# Next up, load whisper medium for english query

# ASR

In [18]:
!pip install -qU transformers

In [19]:
from transformers import pipeline
import torch

In [21]:
if torch.cuda.is_available():
    device = "cuda:0"
    torch_dtype = torch.float16
else:
    device = "cpu"
    torch_dtype = torch.float32

#most likely the audio files will be longer than 30s, set chunking to true
whisper_med = pipeline("automatic-speech-recognition", model="openai/whisper-medium", device=device, torch_dtype=torch_dtype, chunk_length_s=30)


model.safetensors:  15%|#4        | 524M/3.58G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cuda:0


In [22]:
if not os.path.exists('/content/audio'):
    os.makedirs('/content/audio')

In [23]:
import librosa

In [27]:
# load the audio and check the sample rate
sample_audio, _ = librosa.load('/content/audio/Lizard abilities.wav', sr=16000)

In [29]:
sample_transcription = whisper_med(sample_audio)['text']
print(sample_transcription)

/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


 What abilities does the lizard have?


In [30]:
# when you return only the first most relevant chunk, keep in mind the relevant information may span multiple chunks
sample_result = dense_index.search(
    namespace="articles",
    query={
        "top_k": 10,
        "inputs": {
            'text': sample_transcription
        }
    },
    rerank={
      "model": "bge-reranker-v2-m3",
      "top_n": 10,
      "rank_fields": ["chunk_text"]
    }
)
results_filtered = [hit for hit in sample_result['result']['hits'] if hit['_score'] > 0.5]
results_filtered.sort(key=lambda x: (int(x['fields']['document_id'][-1]), x['fields']['chunk_num']))
# print(results_filtered)

chunk_texts = [chunk['fields']['chunk_text'] for chunk in results_filtered]
context = '\n'.join(chunk_texts)
print(context)

Noctiserpens aquavarian: The Shape-Shifting Evolutionary Lizard
Noctiserpens aquavarian is a newly discovered, semi-mythical reptilian species exhibiting remarkable adaptive and morphogenetic abilities. First documented in 2023 in the mangrove-laced coastlines of southeastern Papua New Guinea, this unique lizard is known for its unprecedented capacity to transform into a snake-like form during the night and is showing early-stage biological adaptations toward becoming a fully aquatic marine reptile. Biologists consider it a living example of evolution in motion.

Taxonomy and Classification
	•	Kingdom: Animalia
	•	Phylum: Chordata
	•	Class: Reptilia
	•	Order: Squamata
	•	Family: Noctisauridae (proposed)
	•	Genus: Noctiserpens
	•	Species: N. aquavarian
	•	Common Names: Twilight Shifter, Serpent Lizard, Aqua Wyrm

Appearance
In its diurnal (daytime) form, N. aquavarian resembles a slender, arboreal lizard approximately 50–70 cm in length. Its scales are iridescent, with hues that shift f

# LLM text generation

In [ ]:
# I can use stateless Chatcompletion first. But, then move on to an assistant. Whenever, a user makes a call, a new thread get's created
# I can wait until LLM sends the data. But, then I can use streaming text generation and convert it to speech as it's generated
# I can see the real time trancription by OpenAI


In [25]:
from openai import OpenAI

In [26]:
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [79]:
# from huggingface_hub import notebook_login

# notebook_login()

In [80]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from langchain import PromptTemplate


In [82]:
# vicuna_template = PromptTemplate.from_template(
#     "USER: {question}\n"
#     "CONTEXT:\n{context}\n\n"
#     "ASSISTANT:"
# )

In [31]:
messages = [
    {"role": "system", "content": "You are a helpful assistant. Answer based only on the provided context. If unsure, say 'I don't know.'"},
    {"role": "user",   "content": f"Context:\n{context}\n\nQuestion:\n{sample_transcription}"}
]


In [33]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    temperature=0.5,
    max_tokens=512
)

answer = response.choices[0].message.content
print(answer)

The lizard, Noctiserpens aquavarian, has several abilities, including:
1. Nocturnomorphic shapeshifting between lizard and snake forms.
2. Aquatic pre-adaptation with features like elongated digits, webbed feet, and denser bone structures.
3. Photocamouflage through iridophores for effective camouflage.
4. Neuroadaptive intelligence with spatial memory, problem-solving skills, and mimicry abilities.


In [83]:
# prompt = vicuna_template.format(
#     context=your_context,
#     question=your_question
# )
# prompt

StringPromptValue(text="<s>[INST] <<SYS>>\nYou are a helpful assistant. Answer based solely on the context below.\nIf the answer is not present, say “I don’t have enough information.”\n<</SYS>>\n\nContext:\nNoctiserpens aquavarian: The Shape-Shifting Evolutionary Lizard\nNoctiserpens aquavarian is a newly discovered, semi-mythical reptilian species exhibiting remarkable adaptive and morphogenetic abilities. First documented in 2023 in the mangrove-laced coastlines of southeastern Papua New Guinea, this unique lizard is known for its unprecedented capacity to transform into a snake-like form during the night and is showing early-stage biological adaptations toward becoming a fully aquatic marine reptile. Biologists consider it a living example of evolution in motion.\n\nTaxonomy and Classification\n\t•\tKingdom: Animalia\n\t•\tPhylum: Chordata\n\t•\tClass: Reptilia\n\t•\tOrder: Squamata\n\t•\tFamily: Noctisauridae (proposed)\n\t•\tGenus: Noctiserpens\n\t•\tSpecies: N. aquavarian\n\t•\tC

In [86]:
# def load_chat_model(model_name: str, device: int = 0, dtype=torch.float16):
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model     = AutoModelForCausalLM.from_pretrained(
#         model_name,
#         torch_dtype=dtype
#         )
#     return pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         device=device,
#     )

In [88]:
# # simply try openai's chat completion model
# chat = load_chat_model("lmsys/vicuna-7b-v1.5")
# response = chat(
#     prompt,
#     max_new_tokens=512,
#     do_sample=True,
#     temperature=0.7
# )
# print(response[0]["generated_text"])

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Device set to use cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 56.12 MiB is free. Process 15465 has 14.68 GiB memory in use. Of the allocated memory 14.53 GiB is allocated by PyTorch, and 21.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [85]:
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
# inputs = tokenizer(prompt, return_tensors="pt") # tokenizes the text, transforms into numbers, and returns tensors
# out = model.generate(**inputs, max_new_tokens=1024) # takes in the token ids and generates numbers
# answer = tokenizer.decode(out[0], skip_special_tokens=True) # takes in the token ids and generates text
# answer

# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
# inputs = tokenizer(prompt, return_tensors="pt") # tokenizes the text, transforms into numbers, and returns tensors
# out = model.generate(**inputs, max_new_tokens=1024) # takes in the token ids and generates numbers
# answer = tokenizer.decode(out[0], skip_special_tokens=True) # takes in the token ids and generates text
# answer

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2.
403 Client Error. (Request ID: Root=1-685aa5a1-2b653e216c160a864c6e5150;dc4762e7-5c55-4f2c-8aae-54db2d2c5bda)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.2 is restricted and you are not in the authorized list. Visit https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2 to ask for access.

# TTS pipeline

In [34]:
!pip install -q openai pygame soundfile

In [42]:
import pygame
import io
import soundfile as sf
import tempfile
from IPython.display import Audio, display

In [58]:
# ["alloy", "echo", "fable", "onyx", "nova", "shimmer"]
def TTS(text):
    response = client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=text
    )
    display(Audio(response.content, autoplay=True))
    print("✅ Speech generation completed")

In [59]:
TTS(answer)

✅ Speech generation completed
